<a href="https://colab.research.google.com/github/AbhishekSubramanian/Sentiment-Analysis-IMDB-DistilBert/blob/main/Sentiment_Analysis_IMDB_DistilBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install libraries required for the project
#transformers: HuggingFace library for state-of-the-art NLP models.
#datasets: Easy dataset management.
#evaluate: For model evaluation metrics.
!pip install transformers datasets evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.7 MB/s eta 0:00:00


# Loading the Dataset

In [2]:
# Importing the necessary libraries
from datasets import load_dataset

# Load IMDB dataset from HuggingFace datasets library
dataset = load_dataset("imdb")

# Let's inspect the dataset structure
print(dataset)
print(dataset['train'][0])  # Viewing an example data point


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})
{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and

#Data Preprocessing and Tokenization

In [3]:
# Import DistilBERT tokenizer
from transformers import DistilBertTokenizerFast

# Initialize the tokenizer
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

# Tokenize our dataset
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True, max_length=512)

# Apply tokenization
tokenized_dataset = dataset.map(tokenize, batched=True, batch_size=1000)

# print(f"Keys of tokenized dataset: {list(tokenized_dataset.features)}")
print(tokenized_dataset['train'][0])
# Set dataset format for PyTorch
tokenized_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

# Preparing the data for Training



In [4]:
# Prepare training and evaluation datasets
train_dataset = tokenized_dataset["train"].shuffle(seed=42).select(range(5000)) # 5000 examples for quick training
eval_dataset = tokenized_dataset["test"].shuffle(seed=42).select(range(1000))   # 1000 examples for evaluation


#Define Model and Training Arguments

In [5]:
# Import model and training utilities
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments
import numpy as np
import evaluate

# Load DistilBERT pretrained model for classification
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

# Define metric function for evaluation
accuracy_metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    return accuracy

# Set up training parameters
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=2,              # Training epochs
    per_device_train_batch_size=16,  # Batch size (GPU-dependent)
    per_device_eval_batch_size=16,
    evaluation_strategy="epoch",     # Evaluate at each epoch
    logging_steps=500,
    save_strategy="epoch",
    load_best_model_at_end=True,     # Load best model after training
)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [14]:
# !pip install wandb

import os
os.environ["WANDB_DISABLED"] = "false"


#Training the model

In [15]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

# Start training
trainer.train()


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: abhisub78 (abhisub78-virginia-tech) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.259895,0.891000
2,0.276700,0.323290,0.899000


TrainOutput(global_step=626, training_loss=0.24665988519930612, metrics={'train_runtime': 771.3996, 'train_samples_per_second': 12.963, 'train_steps_per_second': 0.812, 'total_flos': 1324673986560000.0, 'train_loss': 0.24665988519930612, 'epoch': 2.0})

#Evaluating the Model

In [16]:
# Evaluate the trained model on the evaluation dataset
eval_results = trainer.evaluate()

# Print evaluation results
print(f"Evaluation Accuracy: {eval_results['eval_accuracy']:.4f}")


Evaluation Accuracy: 0.8910


#Making Predictions

In [21]:
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
import torch

model_path = "./results/checkpoint-626"

# Load tokenizer and model explicitly
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")
model = DistilBertForSequenceClassification.from_pretrained(model_path)  # path to your trained model

# Function to predict sentiment
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    outputs = model(**inputs)
    probabilities = torch.softmax(outputs.logits, dim=-1)
    prediction = probabilities.argmax().item()
    sentiment = "Positive" if prediction == 1 else "Negative"
    confidence = probabilities[0][prediction].item()
    return sentiment, confidence

# Test the prediction
review = "The movie was great. The acting was good."
sentiment, confidence = predict_sentiment(review)
print(f"Review Sentiment: {sentiment} ({confidence*100:.2f}% confidence)")


Review Sentiment: Positive (99.33% confidence)
